# RA2CE feature: Single link losses analysis

## Single link losses Analysis

In [1]:
from pathlib import Path
from ra2ce.ra2ce_handler import Ra2ceHandler
import geopandas as gpd

# Import config data
root_dir = Path("data", "single_link_losses")
assert root_dir.exists()

# Load network data.
from ra2ce.network.network_config_data.network_config_data_reader import NetworkConfigDataReader
_network_config_data = NetworkConfigDataReader().read(root_dir.joinpath("network.ini"))

# Load analysis data.
from ra2ce.analysis.analysis_config_data.analysis_config_data_reader import AnalysisConfigDataReader
_analysis_config_data = AnalysisConfigDataReader().read(root_dir.joinpath("analysis.ini"))

c:\Users\hauth\anaconda3\envs\ra2ce_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Modify the analysis config data with custom properties

In [2]:
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionLosses

# Modify the first analysis (assuming it's the only one)
_found_analysis: AnalysisSectionLosses = _analysis_config_data.analyses[0]
_input_analysis_path = root_dir.joinpath("input_analysis_data")
_found_analysis.traffic_intensities_file = _input_analysis_path.joinpath("traffic_intensities.csv")
_found_analysis.resilience_curves_file = _input_analysis_path.joinpath("resilience_curve.csv")
_found_analysis.values_of_time_file = _input_analysis_path.joinpath("values_of_time.csv")

### Configure network

Let's have a look first at the hazard overlay to locate the road segments for which losses will be expected. We run only the network configuration.

In [3]:
handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)
handler.configure()

In [4]:
hazard_output = root_dir / "static" / "output_graph" / "base_graph_hazard_edges.gpkg"
hazard_gdf = gpd.read_file(hazard_output, driver = "GPKG")
hazard_gdf.explore(column="EV1_ma", cmap = "viridis")

### Run losses analysis

In [17]:
handler.run_analysis()

c:\Users\hauth\anaconda3\envs\ra2ce_env\lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


[AnalysisResultWrapper(analysis_result=      ID    u    v  key    EV1_fr    EV1_ma  \
 0     86    2   42    0  0.014318  1.369213   
 1     30   29   31    0  0.079975  0.345044   
 2    100   30   82    0  0.100808  0.654458   
 3     85   42   73    0  0.027676  0.409945   
 4    264   46   51    0  0.002634  0.766717   
 ..   ...  ...  ...  ...       ...       ...   
 225  253  153  156    0  0.000000  0.000000   
 226  246  154  155    0  0.000000  0.000000   
 227  249  154  156    0  0.000000  0.000000   
 228  248  156  157    0  0.000000  0.000000   
 229  257  157  158    0  0.000000  0.000000   
 
                                              alt_nodes  alt_time  avgspeed  \
 0    [2, 0, 11, 12, 10, 7, 6, 9, 75, 121, 118, 5, 3...  0.099722      59.0   
 1                                     [29, 30, 82, 31]  0.009356      59.0   
 2                                     [30, 29, 31, 82]  0.005288      30.0   
 3                                                  nan       NaN   

Let's inspect the results of the analysis. For every segment in the network, the losses are expressed in vehicle loss hour (vlh) for all the types of trip defined in the input files: business, commute, freight and other.

In [18]:
analysis_output_folder = root_dir / "output" / "single_link_losses" # specify path to output folder

losses_gdf = gpd.read_file(analysis_output_folder/"beira_single_losses.gpkg") #specify the name of the geopackage holding your results (can be found in the analysis output folder)
losses_gdf.head() #display the attributes of the file

,ID,u,v,key,EV1_fr,EV1_ma,alt_nodes,alt_time,avgspeed,bridge,...,node_B,rfid,rfid_c,time,vlh_business_EV1_ma,vlh_commute_EV1_ma,vlh_freight_EV1_ma,vlh_other_EV1_ma,vlh_EV1_ma_total,geometry
0,86,2,42,0,0.014318,1.369213,"[2, 0, 11, 12, 10, 7, 6, 9, 75, 121, 118, 5, 3...",0.099722,59.0,yes,...,42,125,127,0.043763,433.125000,303.333333,5775.000000,5133.333333,11644.791667,"LINESTRING (34.86702 -19.83164, 34.86711 -19.8..."
1,30,29,31,0,0.079975,0.345044,"[29, 30, 82, 31]",0.009356,59.0,yes,...,31,91,93,0.003898,5.156250,3.750000,5.000000,35.833333,49.739583,"LINESTRING (34.83997 -19.83592, 34.84019 -19.8..."
2,100,30,82,0,0.100808,0.654458,"[30, 29, 31, 82]",0.005288,30.0,yes,...,82,233,241,0.007967,-40.781250,-18.437500,-81.250000,-106.250000,-246.718750,"LINESTRING (34.84149 -19.83431, 34.84137 -19.8..."
3,85,42,73,0,0.027676,0.409945,nan,NaN,60.0,yes,...,73,208,216,0.016917,46.375000,51.187500,24.500000,210.000000,332.062500,"LINESTRING (34.87145 -19.82349, 34.87126 -19.8..."
4,264,46,51,0,0.002634,0.766717,"[46, 48, 47, 160, 51]",0.008829,60.0,nan,...,51,151,153,0.002183,39.785156,13.125000,80.208333,69.270833,202.389323,"LINESTRING (34.85650 -19.83019, 34.85714 -19.8..."


The map below displays the total loss for all trip types per link

In [19]:
losses_roads_map = losses_gdf.explore(column='vlh_EV1_ma_total', tiles="CartoDB positron",)
display(losses_roads_map)